# LLM Utility Package

This package is a simple wrapper for the main LLM providers with the minimal viable set of feature supports (for my workflow). It supports:

1. Concurrent text embeddings (except for Claude)
2. Saving to a chromadb vector store
2. Concurrent chat calls
3. Web search
4. Structured outputs

## 1. Environment Setup

Before running the code, ensure you have a `.env` file in your project root with the following keys:

```bash
GEMINI_TYPE=
GEMINI_PROJECT_ID=
GEMINI_PRIVATE_KEY_ID=
GEMINI_PRIVATE_KEY=
GEMINI_CLIENT_EMAIL=
GEMINI_CLIENT_ID=
GEMINI_AUTH_URI=
GEMINI_TOKEN_URI=
GEMINI_AUTH_PROVIDER_CERT=
GEMINI_CLIENT_CERT_URL=
GEMINI_UNIVERSE_DOMAIN=
ANTHROPIC_API_KEY=
OPENAI_KEY=

```

In [1]:
from dotenv import load_dotenv
import os
import numpy as np
import json

# Load environment variables
load_dotenv()

True

## 2. Initialization

Import and initialize the `LLMWrapper`. This class handles routing requests to the appropriate provider based on the model name.

In [2]:
from llm_utils.wrapper import LLMWrapper

llm = LLMWrapper()

## 3. Basic Chat Completion

The `ask` method is the main entry point. It takes lists of inputs (ids, messages, formats) to concurrent processing, but here we'll show single requests.

In [3]:
system_message = "You are a helpful assistant."
user_messages = [
    "Briefly explain the theory of relativity like I'm five. Then explain it again like I'm a college student.",
    "What is the current price of Tesla's stock? If you can find it, return N/A"
]
response_formats = [
    {
        "name": "example_response1",
        "type": "json_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "FIVE_YEAR_OLD_EXPLANATION": {"type": "string"},
                "COLLEGE_STUDENT_EXPLANATION": {"type": "string"}
            },
            "required": ["FIVE_YEAR_OLD_EXPLANATION", "COLLEGE_STUDENT_EXPLANATION"],
            "additionalProperties": False
        }
    },
    {
        "name": "example_response2",
        "type": "json_schema",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "TICKER": {"type": "string"},
                "CURRENT_PRICE": {"type": "string"},
                "EXCHANGE": {"type": "string"}
            },
            "required": ["TICKER", "CURRENT_PRICE", "EXCHANGE"],
            "additionalProperties": False
        }
    }
]

### OpenAI

In [4]:
response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gpt-5-mini",
    n_workers=2,
)

display(response)

response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gpt-5-mini",
    web_search=True,
    n_workers=2,
)

display(response)

Chat Completion: 100%|██████████| 2/2 [00:13<00:00,  6.56s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': 'Imagine space and time are a big stretchy blanket. When you put a heavy ball on the blanket it makes a dip, and smaller balls roll toward the dip — that’s like gravity pulling things together. Things that move very, very fast (like light) are the fastest and nothing can go faster. When you run really fast, your clock ticks a little slower and your ruler looks a little shorter compared to someone standing still. So space and time can bend and stretch depending on how fast you move and how heavy things are.',
  'COLLEGE_STUDENT_EXPLANATION': 'The theory of relativity has two parts. Special relativity (SR) applies in the absence of gravity and rests on two postulates: the laws of physics are the same in all inertial frames, and the speed of light c is the same for all observers. From these follow the Lorentz transformations, which replace Galilean boosts and lead to relativity of simultaneity, time dilation (moving clocks run slow by factor γ = 1/√(1−v

Chat Completion: 100%|██████████| 2/2 [00:16<00:00,  8.33s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': 'Imagine space is a big, stretchy blanket. When you put something heavy on the blanket (like a planet or the Sun) it makes a dent. Smaller things (like a ball or a moon) roll toward the dent — that’s what we call gravity. Also, light always goes the same fastest speed, like a super-fast car that never slows down. If you could go really, really fast, your clock would tick more slowly than someone who stayed still — but we only notice that if you go almost as fast as light.',
  'COLLEGE_STUDENT_EXPLANATION': "Split the idea into two parts: special relativity and general relativity. \n\nSpecial relativity (1905): Two postulates — the laws of physics are the same in all inertial frames, and the speed of light c is the same for all observers. From these follow the Lorentz transformations, which replace Galilean boosts. Consequences include time dilation (Δt' = γΔt), length contraction (L' = L/γ), relativity of simultaneity, and mass–energy equivalence (E 

### Gemini (Gemini 2.5 Flash)

In [5]:
response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gemini-2.5-flash",
    n_workers=2,
)

display(response)

response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="gemini-2.5-flash",
    web_search=True,
    n_workers=2,
)

display(response)

Chat Completion: 100%|██████████| 2/2 [00:04<00:00,  2.21s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': "Imagine you're running super fast, like a superhero! If you run really, really fast, time for you goes a tiny bit slower than for someone just sitting still. Also, things can look squished if they go super speedy!",
  'COLLEGE_STUDENT_EXPLANATION': 'The theory of relativity, proposed by Albert Einstein, comprises two main theories: special relativity and general relativity. Special relativity postulates that the laws of physics are the same for all non-accelerating observers and that the speed of light in a vacuum is constant for all observers, regardless of their relative motion. This leads to counter-intuitive consequences such as time dilation, length contraction, and the equivalence of mass and energy (E=mc²). General relativity extends this by describing gravity not as a force, but as a curvature of spacetime caused by mass and energy, which affects the path of objects, including light.'},
 '1': {'TICKER': 'TSLA', 'CURRENT_PRICE': '$177.37', 'E

Chat Completion: 100%|██████████| 2/2 [00:16<00:00,  8.36s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': 'Imagine you\'re on a super-fast spaceship! Special Relativity (the fast spaceship part): When your spaceship goes really, really fast, almost as fast as light (which is the fastest anything can go!), some funny things happen. Time inside your spaceship might go a tiny bit slower than for your friends back on Earth. Also, if someone outside looks at your super-fast spaceship, it might look a little bit squished in the direction it\'s moving. But the speed of light itself always looks the same to everyone, no matter how fast they are going. It\'s like a cosmic speed limit that nothing can break. And guess what? Very tiny things can turn into lots and lots of energy, like in superhero explosions! That\'s what E=mc² means. General Relativity (the heavy ball on a trampoline part): Now, imagine you have a big, heavy bowling ball and you put it on a stretchy trampoline. It makes a big dip, right? If you roll a little marble nearby, it won\'t roll in a stra

### Claude (Claude 3.5 Sonnet)

In [6]:
response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="claude-sonnet-4-5",
    n_workers=2,
)

display(response)

response = llm.ask(
    system_message=system_message,
    ids=np.arange(len(user_messages)).astype(str).tolist(),
    user_messages=user_messages,
    response_formats=response_formats,
    model="claude-sonnet-4-5",
    web_search=True,
    n_workers=2,
)

display(response)

Chat Completion: 100%|██████████| 2/2 [00:10<00:00,  5.19s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': "Imagine you're on a super fast train with your toy car. If you roll your toy car forward on the train, it looks normal to you. But to someone standing outside watching the train zoom by, your toy car looks like it's going SUPER duper fast - as fast as the train plus how fast you rolled it! That's kind of what relativity means - things look different depending on where you are and how fast you're moving. Also, time is like a stretchy rubber band - it can go slower or faster depending on how fast you're zooming around! And Einstein discovered that space and time are connected like peanut butter and jelly!",
  'COLLEGE_STUDENT_EXPLANATION': "Einstein's theory of relativity comes in two parts: Special Relativity (1905) and General Relativity (1915). Special Relativity deals with objects moving at constant velocities and introduces two key principles: (1) the laws of physics are the same in all inertial reference frames, and (2) the speed of light in a v

Chat Completion: 100%|██████████| 2/2 [00:11<00:00,  5.98s/it]


{'0': {'FIVE_YEAR_OLD_EXPLANATION': "Imagine you're on a train with your toy car. If you roll the car forward, it looks normal to you. But to someone standing outside watching the train go by, your toy car is moving super fast because it's going the speed of the train PLUS how fast you rolled it! Einstein discovered that time and space are kind of like that train - they're not the same for everyone. If you could go really, really fast (like almost as fast as light), time would actually go slower for you than for your friends at home. Also, really heavy things like the sun are so heavy they bend space around them, kind of like how a bowling ball makes a dent if you put it on a trampoline. That's why planets go around the sun - they're following the bent space!",
  'COLLEGE_STUDENT_EXPLANATION': 'Einstein\'s theory of relativity consists of two parts: Special Relativity (1905) and General Relativity (1915). Special Relativity deals with objects moving at constant velocities and introduce

## Embedding

In [7]:
texts = ["cat", "kit-ty", "dog", "potato"] * 100
ids = np.arange(len(texts)).astype(str).tolist()    

def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

# OpenAI Embeddings
print("Getting OpenAI Embeddings...")
_, openai_embeddings = llm.embed(
    ids=ids,
    texts=texts,
    size=1536,
    db=None,
    name="test_openai_emb",
    verbose=False,
    model="text-embedding-3-small"
)

cat_emb = np.array(openai_embeddings[0])
kitty_emb = np.array(openai_embeddings[1])
dog_emb = np.array(openai_embeddings[2])
potato_emb = np.array(openai_embeddings[3])

sim_cat_kitty = cosine_similarity(cat_emb, kitty_emb)
sim_dog_potato = cosine_similarity(dog_emb, potato_emb)

print(f"OpenAI Similarity (cat, kitty): {sim_cat_kitty:.4f}")
print(f"OpenAI Similarity (dog, potato): {sim_dog_potato:.4f}")

if sim_cat_kitty > sim_dog_potato:
    print("✅ OpenAI Embedding Test Passed: cat-kitty similarity > dog-potato similarity")
else:
    print("❌ OpenAI Embedding Test Failed")

# Gemini Embeddings
print("Getting Gemini Embeddings...")
_, gemini_embeddings = llm.embed(
    ids=ids,
    texts=texts,
    size=768, 
    db=None,
    name="test_gemini_emb",
    verbose=False,
    model="gemini-embedding-001"
)

cat_emb = np.array(gemini_embeddings[0])
kitty_emb = np.array(gemini_embeddings[1])
dog_emb = np.array(gemini_embeddings[2])
potato_emb = np.array(gemini_embeddings[3])

sim_cat_kitty = cosine_similarity(cat_emb, kitty_emb)
sim_dog_potato = cosine_similarity(dog_emb, potato_emb)

print(f"Gemini Similarity (cat, kitty): {sim_cat_kitty:.4f}")
print(f"Gemini Similarity (dog, potato): {sim_dog_potato:.4f}")

if sim_cat_kitty > sim_dog_potato:
    print("✅ Gemini Embedding Test Passed: cat-kitty similarity > dog-potato similarity")
else:
    print("❌ Gemini Embedding Test Failed")

Getting OpenAI Embeddings...
OpenAI Similarity (cat, kitty): 0.4100
OpenAI Similarity (dog, potato): 0.2720
✅ OpenAI Embedding Test Passed: cat-kitty similarity > dog-potato similarity
Getting Gemini Embeddings...
Gemini Similarity (cat, kitty): 0.8416
Gemini Similarity (dog, potato): 0.7459
✅ Gemini Embedding Test Passed: cat-kitty similarity > dog-potato similarity
